# 7장. 가설과 추론

#### 7-1. 통계적 가설 검정
1. 가설(Hypothesis): 'A는 B일 것이다' 라는 형식의 Suggestion.
        - 통계의 관점에서의 가설: 다양한 가정하에서 특정 분포에 대한 확률변수의 관측치.
2. 고전적인 가설 검정의 프로세스 
        - 귀무가설(H0, Null hypothesis): 기본적인 가설 
        - 대립가설(H1, Alternative hypothesis): 비교하고 싶은 가설
          ==> H0과 H1을 통계를 사용해서 비교하고, H0을 기각할지 말지를 결정함.

#### 7-2. 동전 던지기 예시
1. 가정
    - 동전이 공평한 동전인지 아닌지가 검정하고 싶고, 한 동전의 앞면이 나올 확률이 p라고 할 때,
    - 귀무가설(H0): p = 0.5 이다.
    - 대립가설(H1): p != 0.5 이다.
2. 검정 진행
    - Trial: 동전을 n번 던져서 앞면이 나온 횟수 X를 센다.
      * 이 때, 동전던지기는 각각 베르누이 분포를 따름(= X는 이항분포를 따르는 확률변수)

In [4]:
#필요한 모듈, 함수 불러오기 
import math, random

#누적분포함수(Ch6. 확률)
def normal_cdf(x, mu=0,sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

#누적분포함수의 역함수(Ch6. 확률)
def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001): #tolerance = 공차
    """find approximate inverse using binary search"""

    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)

    low_z, low_p = -10.0, 0            # normal_cdf(-10) is (very close to) 0
    hi_z,  hi_p  =  10.0, 1            # normal_cdf(10)  is (very close to) 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2     # consider the midpoint
        mid_p = normal_cdf(mid_z)      # and the cdf's value there
        if mid_p < p:
            # midpoint is still too low, search above it
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            # midpoint is still too high, search below it
            hi_z, hi_p = mid_z, mid_p
        else:
            break

    return mid_z

In [5]:
def normal_approximation_to_binomial(n, p):
    """Binomial(n, p)에 해당되는 평균과 표준편차 계산"""
    mu = p * n  #평균
    sigma = math.sqrt(p * (1 - p) * n) #표준편차
    return mu, sigma

print(normal_approximation_to_binomial(100, 0.5))

(50.0, 5.0)


* 확률변수가 정규분포를 따른다는 가정 하에, normal_cdf를 사용하여 실제 동전 던지기로부터 얻은 값이 구간 안 또는 밖에 존재할 확률을 계산할 수 있다.

In [6]:
#누적분포함수: 확률변수가 특정 값보다 작을 확률을 나타냄.
normal_probability_below = normal_cdf

In [7]:
# it's above the threshold if it's not below the threshold
# 만약 확률변수가 특정 값(threshold)보다 작지 않다면, 특정 값보다 크다는 의미이다.
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

# it's between if it's less than hi, but not less than lo
# 만약 확률변수가 hi보다 작고 lo보다 작지 않다면, 확률변수는 hi와 lo 사이에 존재함.
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# it's outside if it's not between
# 확률변수가 범위 밖에 존재한다면, 범위 안에 존재하지 않다는 의미이다.
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

* 이와 반대로, 확률이 주어졌을 때 평균을 중심으로 하는 대칭적인 구간을 구할 수 있음.
  - 만약 분포의 60%를 차지하는 평균 중심의 구간을 구하고 싶다면, 양쪽 꼬리 부분이 각각 분포의 20%를 차지하는 지점을 구하면 된다.

In [8]:
#만약 분포의 60%를 차지하는 평균 중심의 구간을 구하고 싶다면, 양쪽 꼬리 부분이 각각 분포의 20%를 차지하는 지점을 구하면 된다.

def normal_upper_bound(probability, mu=0, sigma=1):
    """P(Z <= z) = probability인 z 값을 변환"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    """P(Z >= z) = probability인 z 값을 변환"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """입력한 probability(확률)값을 포함하고, 평균을 중심으로 대칭적인 구간을 반환"""
    tail_probability = (1 - probability) / 2

    # upper bound should have tail_probability above it
    # 구간의 상한은 taiL_probability 값 이상의 확률 값을 갖고 있음.
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # lower bound should have tail_probability below it
    # 구간의 하한은 taiL_probability 값 이하의 확률 값을 갖고 있음.
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

* 실제로 동전을 1000번 던질 때(n=1000), 동전이 공평하다는 가설이 맞다면, X는 대략 평균이 500이고 표준편차가 15.8인 정규분포를 따름.

In [9]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

print(mu_0, sigma_0)

500.0 15.811388300841896


* 유의수준과 1종 오류의 개념
    - 유의수준(significance): 1종 오류를 얼마나 허용할지에 대한 한도
    - 1종 오류: 비록 H0가 참이지만 H0를 기각하는 가양성(false positive) 오류.
    - 2종 오류: 비록 H0가 거짓이지만 H0를 채택하는 오류.
    - significance는 보통 5%나 1%로 설정한다.

In [18]:
#1종 오류
#다음 코드로부터 X가 주어진 범위를 벗어나면 귀무가설 H0를 기각하는 가설 검정을 생각해 보자.
normal_two_sided_bounds(0.95, mu_0, sigma_0)

#유의수준은 5%라고 가정해 보자.
#만약 p=0.5라면, 즉 H0가 참이라면 X가 주어진 범위를 벗어날 확률은 5%일 것이다.
#즉, 만약 H0가 참이라면 이 가설검정 중 20번에 19번은 올바른 결과가 나올 것이다.

(469.01026640487555, 530.9897335951244)

In [19]:
#2종 오류

#p가 0.5라고 가정할 때, 유의수준이 5%인 구간
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

#p = 0.55인 경우의 실제 평균과 표준편차
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

#2종오류란 H0을 기각하지 못한다는 의미.
#즉, X가 주어진 구간 안에 존재할 경우를 의미함.
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(power)

0.8865480012953671


* 단측검정이 필요한 경우
    -  p <= 0.5, 즉 동전이 앞면에 편향되지 않은 경우를 H0로 정한다면?
    - X > 50이면 H0 기각, X < 50이면 기각하지 않는다(단측검정)
    - 유의수준이 5%인 가설검정을 위해서는 normal_probability_below를 사용하여 분포의 95%가 해당 값 이하인 경계 값을 찾을 수 있다.

In [24]:
hi = normal_upper_bound(0.95, mu_0, sigma_0) #
print(hi) # hi = 526( < 531, 분포 상위 부분에 더 높은 확률을 주기 위해서)

type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability   #0.936
print(power)


#X가 469보다 작을 때 H0를 기각하는 게 아니라(H1이 참이라면 이는 거의 발생하지 않음),
#X가 526 ~ 531일 때 H0를 기각하기 때문에(H1이 참이라면 이는 가끔 발생함) 전보다 검정력이 더 좋아졌다고 볼 수 있음.

526.0073585242053
0.9363794803307173


#### 7-3. p-value